In [75]:
import requests
import pandas as pd
import datetime
import quantstats as qs

def get_symphony_backtest(symphony_id, baseline_id='', starting_money=10000):
    payload = {
        'capital': starting_money,
        'apply_reg_fee': True,
        'apply_taf_fee': True,
        'backtest_version': "v2",
        'slippage_percent': 0.0005,
        'broker': 'apex'
    }
    if baseline_id:
        payload['benchmark_symphonies'] = [baseline_id]
    url = f"https://backtest-api.composer.trade/api/v2/public/symphonies/{symphony_id}/backtest"
    res = requests.post(url, json=payload)
    res.raise_for_status()
    return res.json()

def get_percent_close(symphony_id, results, starting_money=10000):
    prev = starting_money
    first_date = datetime.datetime.strptime("01/01/1970", "%m/%d/%Y") + datetime.timedelta(days=int(results['first_day']))
    day_col = []
    percent_col = []
    for day in sorted(results['dvm_capital'][symphony_id].keys()):
        dt = datetime.datetime.strptime("01/01/1970", "%m/%d/%Y") + datetime.timedelta(days=int(day))
        percent = ((results['dvm_capital'][symphony_id][day] - prev) / prev)
        percent = 0 if dt == first_date else percent
        day_col.append(dt)
        percent_col.append(percent)
        prev = results['dvm_capital'][symphony_id][day]
    df = pd.DataFrame({'percent': percent_col})
    df.index = day_col
    return df

def cagr_sans_2020(returns, periods=365):
    returns = returns.drop(returns.loc['2020-01-01':'2021-01-01'].index)
    total = returns.add(1).prod() - 1
    years = ((returns.index[-1] - returns.index[0]).days / periods) - 1 # -1 from removing 2020
    res = abs(total + 1.0) ** (1.0 / years) - 1
    return res.iloc[0]

def create_stats_dict(name, stats, close):
    return {'Name': name,
            'AR': stats['annualized_rate_of_return'],
            'AR x-2020': cagr_sans_2020(close),
            'Sharpe': stats['sharpe_ratio'],
            'SD': stats['standard_deviation'],
            'Calmar': stats['calmar_ratio'],
            'MD': stats['max_drawdown'],
            'Win %': qs.stats.win_rate(close).iloc[0],
            'Recovery': qs.stats.recovery_factor(close).iloc[0],
            '1M': stats['trailing_one_month_return']}

def get_results(symphony_id, baseline_id='', starting_money=10000, max_name_length=25):
    if baseline_id:
        results = get_symphony_backtest(symphony_id, baseline_id, starting_money)
        df1 = get_percent_close(symphony_id, results, starting_money)
        stats1 = create_stats_dict(results['legend'][symphony_id]['name'][:max_name_length], results['stats'], df1)
        df2 = get_percent_close(baseline_id, results, starting_money)
        stats2 = create_stats_dict(results['legend'][baseline_id]['name'][:max_name_length], results['stats']['benchmarks'][baseline_id], df2)
        results = pd.DataFrame([stats1, stats2])
        results.index = results['Name']
        results = results.drop(columns='Name')
        diff = results.diff(-1).dropna()
        diff.index = ['Difference']
        results = pd.concat([results, diff], sort=False)
    else:
        results = get_symphony_backtest(symphony_id, starting_money=starting_money)
        df = get_percent_close(symphony_id, results, starting_money)
        stats = create_stats_dict(results['legend'][symphony_id]['name'], results['stats'], df)
        results = pd.DataFrame([stats])
        results.index = results['Name']
        results = results.drop(columns='Name')
    return results.style.format({'AR': "{:.2%}",'AR x-2020': "{:.2%}",'Sharpe': "{:.3f}",'SD': "{:.2%}",'Calmar': "{:.2f}",'MD': "{:.2%}",'Win %': "{:.2%}",'Recovery': "{:.2f}",'1M': "{:.2%}"})

In [76]:
symphony_id = 'OPYgPFouAcCSonKKj54e'
baseline_id = '8HhxQ9Iq84VTULMP89Uz'
get_results(symphony_id, baseline_id)

,AR,AR x-2020,Sharpe,SD,Calmar,MD,Win %,Recovery,1M
V3.1 Anansi Portfolio: 75,444.01%,255.47%,3.938,45.57%,24.78,17.92%,62.63%,59.44,12.74%
Live Portfolio,413.63%,228.47%,3.814,45.53%,23.79,17.39%,61.43%,59.30,10.82%
Difference,30.39%,27.00%,0.123,0.04%,0.99,0.53%,1.20%,0.15,1.93%
